# minimal

In [2]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [154]:
# solve after 40s of training 5000/128
# default_maze = torch.tensor([
#     [1, 0, 0, 0, 0],
#     [1, 1, 1, 1, 1],
#     [0, 1, 0, 1, 0],
#     [1, 1, 0, 0, 0],
#     [1, 1, 1, 1, -1],
# ])

default_maze = torch.tensor([
    [1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0],
    [1, 1, 1, 1, -1],
])

# default_maze = torch.tensor([
#     [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#     [0, 1, 1, 1, 1, 1, 1, 0, 0, 0],
#     [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
#     [0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
#     [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
#     [0, 1, 1, 1, 1, 1, 1, 0, 0, 1],
#     [0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
#     [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
#     [0, 0, 0, 0, 1, 1, 1, 1, 1, -1],
# ])

# doesn't solve even after 2h of training 100000/512
# default_maze = torch.tensor([
#     [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
#     [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1],
#     [1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0],
#     [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0],
#     [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
#     [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
#     [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
#     [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0],
#     [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
#     [0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0],
#     [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
#     [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#     [1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#     [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
#     [1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0],
#     [0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
#     [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, -1],
# ])
MAZE_WIDTH = default_maze.shape[0]
INPUT_SIZE = MAZE_WIDTH * MAZE_WIDTH + 2 * MAZE_WIDTH
MOVES = {
    (-1, 0): torch.tensor(0).to(device), # up
    (1, 0):  torch.tensor(1).to(device), # down
    (0, -1): torch.tensor(2).to(device), # left
    (0, 1):  torch.tensor(3).to(device),  # right
 }

# policy
HIT_WALL_PENALTY = -1
MOVE_PENALTY = 0
WIN_REWARD = 10
 
# hyperparams
# BATCH_SIZE = 128
EPOCH = 10000
BATCH_SIZE = 512
# EPOCH = 100000
# EPOCH = 5000
LEARNING_RATE = 3e-4

def get_maze():
    maze = default_maze
    rewards = torch.zeros_like(maze)
    rewards[maze == 0] = HIT_WALL_PENALTY
    rewards[maze == 1] = MOVE_PENALTY
    rewards[maze == -1] = WIN_REWARD
    return maze, rewards

def get_reward(rewards, pos):
    x, y = pos
    a, b = rewards.shape
    if 0 <= x < a and 0 <= y < b:
        return rewards[x, y]
    return HIT_WALL_PENALTY

def get_next_pos(maze, rewards, pos):
    new_pos = pos # default to bouncing off a wall.
    reward = HIT_WALL_PENALTY # default to hitting a wall.
    move = random.choice(list(MOVES.keys()))
    x, y = pos
    a, b = maze.shape
    i, j = move
    if 0 <= x + i < a and 0 <= y + j < b:
        # if maze[x + i, y + j] != -1:
        new_pos = (x + i, y + j)
        reward = get_reward(rewards, new_pos)
    return new_pos, reward, move

# def get_batch():
#     batch = []
#     maze, rewards = get_maze()
#     positions = random.choices((maze == 1).nonzero().tolist(), k=BATCH_SIZE)
#     for pos in positions:
#         new_pos, reward, move = get_next_pos(maze, rewards, pos)
#         batch.append((pos, move, new_pos, reward))
#     return maze, batch

def get_batch():
    batch = []
    maze, rewards = get_maze()
    for pos in (maze == 1).nonzero().tolist():
    # for pos in positions:
        new_pos, reward, move = get_next_pos(maze, rewards, pos)
        batch.append((pos, move, new_pos, reward))
    return maze, batch

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(INPUT_SIZE, INPUT_SIZE),
            nn.LayerNorm(INPUT_SIZE),
            nn.ReLU(),
            nn.Linear(INPUT_SIZE, INPUT_SIZE),
            nn.ReLU(),
            nn.Linear(INPUT_SIZE, INPUT_SIZE),
            nn.ReLU(),
            nn.Linear(INPUT_SIZE, len(MOVES)),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)        
        return logits

def to_input(maze, pos):
    return torch.cat((
        maze.view(-1),
        F.one_hot(torch.tensor(pos), num_classes=MAZE_WIDTH).view(-1),
    )).float().to(device)

def train(model):
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    losses = []
    for epoch in range(EPOCH):
        maze, batch = get_batch()

        # train vectorized
        # ----------------
        xs, ms, ys, rs, nuke = [], [], [], [], []
        for pos, move, new_pos, reward in batch:
            xs.append(to_input(maze, pos))
            ms.append(F.one_hot(MOVES[move], num_classes=len(MOVES)))
            ys.append(to_input(maze, new_pos))
            rs.append(reward)
            nuke.append(0. if reward == -1 else 1.)

        XS = torch.stack(xs).to(device)
        MS = torch.stack(ms).to(device)
        YS = torch.stack(ys).to(device)
        RS = torch.tensor(rs).to(device).view(-1, 1)
        NUKE = torch.tensor(nuke).to(device).view(-1, 1)
        bellman_left = (model(XS) * MS).sum(dim=1, keepdim=True)
        qqs = model(YS).max(dim=1, keepdim=True).values

        decay = 0.9
        bellman_right = RS + qqs * NUKE * decay

        # Bellman equation
        # Q(m, p) = r + max(Q(m', p'))
        loss = F.mse_loss(bellman_left, bellman_right)
        losses.append(loss.item())
        if epoch % 50 == 49:
            print(f"epoch: {epoch: 5} loss: {torch.tensor(losses).mean():.8f}")
            losses = []
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # train non-vectorized
        # --------------------
        # lefts, rights = [], []
        # for pos, move, new_pos, reward in batch:
        #     qs = model(to_input(maze, pos))
        #     hot = F.one_hot(MOVES[move], num_classes=len(MOVES))
        #     bellman_left = (qs * hot).sum()

        #     new_q = model(to_input(maze, new_pos)).max()
        #     bellman_right = reward + new_q

        #     lefts.append(bellman_left)
        #     rights.append(bellman_right)
        
        # bellman_left = torch.stack(lefts).to(device)
        # bellman_right = torch.stack(rights).to(device)
        # loss = F.mse_loss(bellman_left, bellman_right)
        # losses.append(loss.item())
        # if epoch % 50 == 0:
        #     print(f"epoch: {epoch: 5} loss: {torch.tensor(losses).mean():.8f}")
        #     losses = []
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

model = NeuralNetwork().to(device)
train(model)

epoch:    49 loss: 1.95360065
epoch:    99 loss: 1.89844728
epoch:   149 loss: 1.88555324
epoch:   199 loss: 1.46310210
epoch:   249 loss: 1.20909953
epoch:   299 loss: 0.65845472
epoch:   349 loss: 1.01334047
epoch:   399 loss: 0.88463944
epoch:   449 loss: 0.50584602
epoch:   499 loss: 0.41929311
epoch:   549 loss: 0.34425122
epoch:   599 loss: 0.34975883
epoch:   649 loss: 0.29601127
epoch:   699 loss: 0.25800046
epoch:   749 loss: 0.18110007
epoch:   799 loss: 0.17918137
epoch:   849 loss: 0.15019381
epoch:   899 loss: 0.21093020
epoch:   949 loss: 0.21180575
epoch:   999 loss: 0.14081015
epoch:  1049 loss: 0.16018997
epoch:  1099 loss: 0.14671925
epoch:  1149 loss: 0.14184570
epoch:  1199 loss: 0.13095742
epoch:  1249 loss: 0.14048429
epoch:  1299 loss: 0.09658053
epoch:  1349 loss: 0.09333814
epoch:  1399 loss: 0.08692633
epoch:  1449 loss: 0.09685785
epoch:  1499 loss: 0.07843018
epoch:  1549 loss: 0.06664760
epoch:  1599 loss: 0.07026114
epoch:  1649 loss: 0.06158974
epoch:  16

In [159]:
i2move = {i.detach().item(): v for v, i in MOVES.items()}

def play(model, maze, pos=(0, 0)):
    print(maze)
    depth = 1000
    while True:
        qs = model(to_input(maze, pos))
        print(f'{qs=}')
        move = i2move[qs.argmax().tolist()]
        new_pos = (pos[0] + move[0], pos[1] + move[1])
        print(f'chose {move} from {pos} to {new_pos}')
        if 0 <= new_pos[0] < MAZE_WIDTH and 0 <= new_pos[1] < MAZE_WIDTH:
            pos = new_pos
            if maze[pos] == -1:
                print("WIN")
                break
            elif maze[pos] == 0:
                print("LOSE: HIT WALL")
                break
        else:
            print("LOSE: OUTSIDE MAZE")
            break
        depth -= 1
        if depth == 0:
            print("LOSE: TOO DEEP")
            break

play(model, default_maze, pos=(0, 0))

# print(default_maze)
# qs = model(to_input(default_maze, (4, 3)))
# print(f'{qs=}')

tensor([[ 1,  1,  1,  1,  1],
        [ 0,  0,  0,  0,  1],
        [ 1,  1,  1,  1,  1],
        [ 1,  0,  0,  0,  0],
        [ 1,  1,  1,  1, -1]])
qs=tensor([-0.9993, -1.0012, -0.9986,  0.0546], device='cuda:0',
       grad_fn=<AddBackward0>)
chose (0, 1) from (0, 0) to (0, 1)
qs=tensor([-0.9982, -0.9997,  0.0492,  0.0627], device='cuda:0',
       grad_fn=<AddBackward0>)
chose (0, 1) from (0, 1) to (0, 2)
qs=tensor([-0.9999, -0.9990,  0.0571,  0.0711], device='cuda:0',
       grad_fn=<AddBackward0>)
chose (0, 1) from (0, 2) to (0, 3)
qs=tensor([-0.9990, -1.0003,  0.0651,  0.0800], device='cuda:0',
       grad_fn=<AddBackward0>)
chose (0, 1) from (0, 3) to (0, 4)
qs=tensor([-0.9981,  0.0892,  0.0725, -0.9993], device='cuda:0',
       grad_fn=<AddBackward0>)
chose (1, 0) from (0, 4) to (1, 4)
qs=tensor([ 0.0819,  0.0997, -1.0012, -1.0003], device='cuda:0',
       grad_fn=<AddBackward0>)
chose (1, 0) from (1, 4) to (2, 4)
qs=tensor([ 0.0947, -0.9999,  0.1173, -0.9979], device='cuda:0'

In [137]:
[x for x in get_batch()[1] if x[0] == [4, 3]]

[([4, 3], (0, -1), (4, 2), tensor(0))]

In [149]:
train(model)

epoch:    49 loss: 0.00001143
epoch:    99 loss: 0.00000130
epoch:   149 loss: 0.00000149
epoch:   199 loss: 0.00000105
epoch:   249 loss: 0.00000118
epoch:   299 loss: 0.00000364
epoch:   349 loss: 0.00000395
epoch:   399 loss: 0.00002352
epoch:   449 loss: 0.00000635
epoch:   499 loss: 0.00000311
epoch:   549 loss: 0.00000076
epoch:   599 loss: 0.00000031
epoch:   649 loss: 0.00000027
epoch:   699 loss: 0.00000153
epoch:   749 loss: 0.00000111
epoch:   799 loss: 0.00000043
epoch:   849 loss: 0.00000029
epoch:   899 loss: 0.00000078
epoch:   949 loss: 0.00000059
epoch:   999 loss: 0.00000046


In [84]:
# backup to disk
# torch.save(model.state_dict(), 'default-maze.pt')

# m = NeuralNetwork()
# m.load_state_dict(torch.load('bigger-maze.pt'))

RuntimeError: Error(s) in loading state_dict for NeuralNetwork:
	Missing key(s) in state_dict: "linear_relu_stack.2.weight", "linear_relu_stack.2.bias", "linear_relu_stack.4.weight", "linear_relu_stack.4.bias", "linear_relu_stack.6.weight", "linear_relu_stack.6.bias". 
	Unexpected key(s) in state_dict: "linear_relu_stack.7.weight", "linear_relu_stack.7.bias", "linear_relu_stack.1.weight", "linear_relu_stack.1.bias", "linear_relu_stack.3.weight", "linear_relu_stack.3.bias", "linear_relu_stack.5.weight", "linear_relu_stack.5.bias". 
	size mismatch for linear_relu_stack.0.weight: copying a param with shape torch.Size([323, 323]) from checkpoint, the shape in current model is torch.Size([35, 35]).
	size mismatch for linear_relu_stack.0.bias: copying a param with shape torch.Size([323]) from checkpoint, the shape in current model is torch.Size([35]).